# Amazon Products Dataset – Data Cleaning & Preprocessing

This notebook demonstrates the steps to clean and preprocess the **Amazon Products Sales Dataset**.

**Objectives:**
1. Load raw dataset downloaded via KaggleHub.
2. Normalize numeric fields (ratings, reviews, prices, purchases).
3. Extract and clean categorical and boolean information (best seller, sponsored, coupon).
4. Parse delivery details and sustainability badges.
5. Convert timestamp fields to datetime.
6. Save cleaned dataset in a portable format.


In [1]:
import kagglehub
import pandas as pd
import re
import datetime
from pathlib import Path

### Step 0: Download Dataset 
We download the dataset using KaggleHub and identify the path to the CSV file.

In [2]:
path = kagglehub.dataset_download(
    "ikramshah512/amazon-products-sales-dataset-42k-items-2025"
)
print("Path to dataset files:", path)

dataset_path = Path(path) / "amazon_products_sales_data_uncleaned.csv"
project_folder = Path.cwd().parent
cleaned_folder = project_folder / "cleaned_data"
cleaned_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist
output_path_full = cleaned_folder / "cleaned_amazon_products_ds.csv"
output_path_sample = cleaned_folder / "sample_amazon_products.csv"

Path to dataset files: C:\Users\dahyu\.cache\kagglehub\datasets\ikramshah512\amazon-products-sales-dataset-42k-items-2025\versions\1


### Step 1: Load Raw Dataset
Load the CSV into a pandas DataFrame. Using Pathlib for portability.

In [3]:
df = pd.read_csv(dataset_path)
df.head(5)

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


### Step 2: Normalize Numeric Fields
We clean and convert numeric data for analysis:
- Ratings → float
- Number of reviews → int
- Purchases in last month → int
- Prices → float

In [4]:
# Ratings
df['rating'] = (
    df['rating'].fillna(0).astype(str).str.extract(r'([\d.]+)').astype(float)
)

# Number of reviews
df['number_of_reviews'] = (
    df['number_of_reviews'].fillna(0).astype(str).str.replace(',', '', regex=False).astype(int)
)

# Bought in last month
def parse_bought(x):
    if pd.isna(x) or 'bought in past month' not in str(x):
        return 0
    x = str(x).replace('+', '').strip()
    match = re.search(r'([\d\.]+)([KM]?)\s*bought in past month', x, re.IGNORECASE)
    if match:
        num, suffix = match.groups()
        num = float(num)
        if suffix.upper() == 'K':
            return int(num * 1_000)
        elif suffix.upper() == 'M':
            return int(num * 1_000_000)
        return int(num)
    return 0

df['bought_in_last_month'] = df['bought_in_last_month'].apply(parse_bought)

# Current/discounted price
df['current/discounted_price'] = (
    df['current/discounted_price'].astype(str).str.replace(',', '', regex=False).astype(float)
)

# Variant price
def parse_variant_price(x):
    if pd.isna(x):
        return 0
    match = re.search(r'basic variant price:\s*\$(\d+(?:\.\d+)?)', str(x), re.IGNORECASE)
    return float(match.group(1).replace(',', '')) if match else 0

df['price_on_variant'] = df['price_on_variant'].apply(parse_variant_price)

# Listed price
def parse_listed_price(x):
    if pd.isna(x):
        return 0
    match = re.search(r'\$(\d+(?:\.\d+)?)', str(x), re.IGNORECASE)
    return float(match.group(1).replace(',', '')) if match else 0

df['listed_price'] = df['listed_price'].apply(parse_listed_price)

### Step 3: Best Seller, Sponsored, and Coupon Fields
We standardize categorical fields into booleans or numeric values for analysis.

In [5]:
# Best Seller & Sponsored
df['is_best_seller'] = df['is_best_seller'].apply(lambda x: str(x).strip() == 'Best Seller')
df['is_sponsored'] = df['is_sponsored'].apply(lambda x: str(x).strip() == 'Sponsored')

def coupon_label(x):
    x = str(x).strip()
    if '%' in x:
        return x  # leave % values as-is
    elif '$' in x:
        return x  # leave $ values as-is
    else:
        return '0%'

df['coupon_label'] = df['is_couponed'].apply(coupon_label)


### Step 4: Delivery Information
Extract detailed delivery information including cost, free delivery and weekday delivery.

In [6]:
def delivery_categories(x):
    if pd.isna(x):
        return pd.Series([0.0, False, None])

    x_str = str(x)

    # Delivery cost
    match_cost = re.search(r'\$(\d+(?:\.\d+)?)', x_str)
    delivery_cost = float(match_cost.group(1)) if match_cost else 0.0

    # Free delivery
    is_free = 'FREE delivery' in x_str

    # Weekday
    weekday_match = re.search(r'(Mon|Tue|Wed|Thu|Fri|Sat|Sun)', x_str)
    weekday = weekday_match.group(1) if weekday_match else None

    return pd.Series([delivery_cost, is_free, weekday])

# Apply updated parsing
df[['delivery_cost', 'is_free_delivery', 'delivery_weekday']] = df['delivery_details'].apply(delivery_categories)

### Step 5 & 6: Drop Unused Columns and Parse Sustainability Badges
We remove irrelevant columns and split sustainability badges into main and extra counts.

In [7]:
df['sustainability_badges'] = df['sustainability_badges'].fillna('None')

def parse_sustainability(x):
    extra_match = re.search(r'\+(\d+) more', x)
    extra_count = int(extra_match.group(1)) if extra_match else 0
    main_badge = x.split('+')[0].strip()
    return pd.Series([main_badge, extra_count])

df[['main_sustainability_badge', 'extra_badges_count']] = df['sustainability_badges'].apply(parse_sustainability)

df.drop(columns=['is_couponed', 'delivery_details', 'sustainability_badges','image_url','product_url','buy_box_availability'], inplace=True)


### Step 7: Convert Collected Timestamp

In [8]:
df['collected_at'] = pd.to_datetime(df['collected_at'])

### Step 8: Inspect DataFrame
Check structure, datatypes, and preview sample data.

In [9]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   title                      42675 non-null  object        
 1   rating                     42675 non-null  float64       
 2   number_of_reviews          42675 non-null  int64         
 3   bought_in_last_month       42675 non-null  int64         
 4   current/discounted_price   30926 non-null  float64       
 5   price_on_variant           42675 non-null  float64       
 6   listed_price               42675 non-null  float64       
 7   is_best_seller             42675 non-null  bool          
 8   is_sponsored               42675 non-null  bool          
 9   collected_at               42675 non-null  datetime64[ns]
 10  coupon_label               42675 non-null  object        
 11  delivery_cost              42675 non-null  float64       
 12  is_f

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,collected_at,coupon_label,delivery_cost,is_free_delivery,delivery_weekday,main_sustainability_badge,extra_badges_count
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375,300,89.68,0.00,159.00,False,True,2025-08-21 11:14:29,Save 15% with coupon,0.0,False,Mon,Carbon impact,0
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457,6000,9.99,0.00,15.99,False,True,2025-08-21 11:14:29,0%,0.0,False,Fri,None,0
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044,2000,314.00,0.00,349.00,False,True,2025-08-21 11:14:29,0%,0.0,False,Mon,None,0
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882,10000,NaN,162.24,0.00,True,False,2025-08-21 11:14:29,0%,0.0,False,None,None,0
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988,10000,NaN,72.74,0.00,False,False,2025-08-21 11:14:29,0%,0.0,False,None,None,0


### Step 9: Save Cleaned Dataset
Save the cleaned data to a portable CSV file.

In [10]:
df.to_csv(output_path_full, index=False)  # full dataset (local only)
df.head(1000).to_csv(output_path_sample, index=False)  # smaller sample for GitHub

print(f"Full dataset saved at: {output_path_full}")
print(f"Sample dataset saved at: {output_path_sample}")

Full dataset saved at: c:\Users\dahyu\OneDrive\Desktop\Projects\amazon_product_analysis\cleaned_data\cleaned_amazon_products_ds.csv
Sample dataset saved at: c:\Users\dahyu\OneDrive\Desktop\Projects\amazon_product_analysis\cleaned_data\sample_amazon_products.csv
